# Dask Arrays
     
Dask arrays coordinate many Numpy arrays, arranged into chunks within a grid.  They support a large subset of the Numpy API.

## Numpy-like operations on Dask array

Le's create a 10000x10000 array of random numbers, represented as many numpy arrays of size 1000x1000 (or smaller if the array cannot be divided evenly). In this case there are 100 (10x10) numpy arrays of size 1000x1000.

In [16]:
import dask.array as da
x = da.random.random((10000, 10000), chunks=(1000, 1000))
x

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

Use NumPy syntax as usual

In [17]:
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z

dask.array<mean_agg-aggregate, shape=(5000,), dtype=float64, chunksize=(500,), chunktype=numpy.ndarray>

Call `.compute()` when you want your result as a NumPy array.

In [18]:
z.compute()

array([0.99062178, 0.99873727, 0.99835523, ..., 0.9917731 , 1.01174507,
       0.99875315])

## Persist data in memory

If you have the available RAM for your dataset then you can persist data in memory.  This allows future computations to be much faster.
Note that this is only relevant if you are in a distributed environment. On a local machine (using single-machine schedulers) `persist` just triggers immediate computation like `compute`.

In [19]:
y = y.persist()

In [20]:
%time y[0, 0].compute()

CPU times: user 1.11 ms, sys: 0 ns, total: 1.11 ms
Wall time: 1.12 ms


np.float64(0.829805894690167)

In [21]:
%time y.sum().compute()

CPU times: user 157 ms, sys: 8.94 ms, total: 166 ms
Wall time: 46.8 ms


np.float64(100019394.30443288)

## Stack, Concatenate, and Block

Often we have many arrays stored on disk that we want to stack together and think of as one large array. To solve this problem, we use the functions `da.stack`, `da.concatenate`, and `da.block`.

### Stack
We stack many existing Dask arrays into a new array, creating a new dimension as we go.

In [22]:
import dask.array as da

arr0 = da.random.random((3, 4), chunks=(1, 2))
arr1 = da.random.random((3, 4), chunks=(1, 2))

data = [arr0, arr1]

In [23]:
arr0

dask.array<random_sample, shape=(3, 4), dtype=float64, chunksize=(1, 2), chunktype=numpy.ndarray>

In [24]:
arr1

dask.array<random_sample, shape=(3, 4), dtype=float64, chunksize=(1, 2), chunktype=numpy.ndarray>

In [25]:
da.stack(data, axis=0)

dask.array<stack, shape=(2, 3, 4), dtype=float64, chunksize=(1, 1, 2), chunktype=numpy.ndarray>

In [26]:
da.stack(data, axis=1)

dask.array<stack, shape=(3, 2, 4), dtype=float64, chunksize=(1, 1, 2), chunktype=numpy.ndarray>

In [27]:
da.stack(data, axis=-1)

dask.array<stack, shape=(3, 4, 2), dtype=float64, chunksize=(1, 2, 1), chunktype=numpy.ndarray>

### Concatenate
We concatenate existing arrays into a new array, extending them along an existing dimension

In [28]:
import dask.array as da

arr0 = da.random.random((3, 4), chunks=(1, 2))
arr1 = da.random.random((3, 4), chunks=(1, 2))

data = [arr0, arr1]

In [29]:
da.concatenate(data, axis=0)

dask.array<concatenate, shape=(6, 4), dtype=float64, chunksize=(1, 2), chunktype=numpy.ndarray>

In [30]:
da.concatenate(data, axis=1)

dask.array<concatenate, shape=(3, 8), dtype=float64, chunksize=(1, 2), chunktype=numpy.ndarray>

### Block 
We can handle a larger variety of cases with `da.block` as it allows concatenation to be applied over multiple dimensions at once. This is useful if your chunks tile a space, for example if small squares tile a larger 2-D plane..

In [31]:
import dask.array as da
import numpy as np

arr0 = da.random.random((3, 4), chunks=(1, 2))
arr1 = da.random.random((3, 4), chunks=(1, 2))

data = [
    [arr0, arr1],
    [arr1, arr0]
]

In [32]:
arr0

dask.array<random_sample, shape=(3, 4), dtype=float64, chunksize=(1, 2), chunktype=numpy.ndarray>

In [33]:
arr1

dask.array<random_sample, shape=(3, 4), dtype=float64, chunksize=(1, 2), chunktype=numpy.ndarray>

In [34]:
da.block(data)

dask.array<concatenate, shape=(6, 8), dtype=float64, chunksize=(1, 2), chunktype=numpy.ndarray>

## Get to know the chunks

If you have a Dask array and want to know more information about chunks and their size, you can use the `chunksize` and `chunks` attributes to access this information.

We always specify a chunks argument to tell `dask.array` how to break up the underlying array into chunks. We can specify chunks in a variety of ways: 
- A uniform dimension size like `1000`, meaning chunks of size `1000` in each dimension 
- A uniform chunk shape like `(1000, 2000, 3000)`, meaning chunks of size `1000` in the first axis, `2000` in the second axis, and 3000 in the third 
- Fully explicit sizes of all blocks along all dimensions, like `((1000, 1000, 500), (400, 400), (5, 5, 5, 5, 5))` 
- A dictionary specifying chunk size per dimension like `{0: 1000, 1: 2000, 2: 3000}`. This is just another way of writing the forms 2 and 3 above

Chunks may include three special values:
- `-1` : no chunking along this dimension
- `None` : no change to the chunking along this dimension (useful for rechunk)
- `"auto"` : allow the chunking in this dimension to accommodate ideal chunk sizes

In [35]:
darr = da.random.random((1000, 1000, 1000))
darr

dask.array<random_sample, shape=(1000, 1000, 1000), dtype=float64, chunksize=(255, 255, 255), chunktype=numpy.ndarray>

In [36]:
darr.chunksize

(255, 255, 255)

In [37]:
darr.chunks

((255, 255, 255, 235), (255, 255, 255, 235), (255, 255, 255, 235))

Sometimes you need to change the chunking layout of your data. For example, perhaps it comes to you chunked row-wise, but you need to do an operation that is much faster if done across columns. You can change the chunking with the rechunk method.  sizes

In [38]:
darr = darr.rechunk([100, None, None])

In [39]:
darr

dask.array<rechunk-merge, shape=(1000, 1000, 1000), dtype=float64, chunksize=(100, 255, 255), chunktype=numpy.ndarray>

In [40]:
darr.chunksize

(100, 255, 255)

In [41]:
darr.chunks

((100, 100, 100, 100, 100, 100, 100, 100, 100, 100),
 (255, 255, 255, 235),
 (255, 255, 255, 235))

## Operate with blocks

`dask.array.Array.blocks` offers an array-like interface to the blocks of an array. This returns a Blockview object that provides an array-like interface to the blocks of a dask array. Numpy-style indexing of a Blockview object returns a selection of blocks as a new dask array. You can index `array.blocks` like a numpy array of shape equal to the number of blocks in each dimension, (available as `array.blocks.size`).

In [42]:
x = da.arange(8, chunks=2)
x

dask.array<arange, shape=(8,), dtype=int64, chunksize=(2,), chunktype=numpy.ndarray>

In [43]:
x.blocks

In [44]:
x.blocks.size

4

In [45]:
x.blocks.shape # aliases x.numblocks

(4,)

In [46]:
x.numblocks

(4,)

In [47]:
x.blocks[0].compute()

array([0, 1])

In [48]:
x.blocks[:3].compute()

array([0, 1, 2, 3, 4, 5])

In [49]:
x.blocks[::2].compute()

array([0, 1, 4, 5])

In [50]:
x.blocks[[-1, 0]].compute()

array([6, 7, 0, 1])

## Dask arrays from different sources

Create dask array from something that looks like an array.

Input must have a .shape, .ndim, .dtype and support numpy-style slicing.

In [51]:
import numpy as np
a = da.from_array(np.array([[1, 2], [3, 4]]), chunks=(1,1))
a

dask.array<array, shape=(2, 2), dtype=int64, chunksize=(1, 1), chunktype=numpy.ndarray>

You can create a dask array from a dask delayed value (this routine is useful for constructing dask arrays in an ad-hoc fashion using dask delayed, particularly when combined with stack and concatena).

The dask array will consist of a single chunk.

In [52]:
import dask.delayed as dd
import dask.array as da
import numpy as np
value = dd(np.ones)(5)
array = da.from_delayed(value, (5,), dtype=float)
array.compute()
array

dask.array<from-value, shape=(5,), dtype=float64, chunksize=(5,), chunktype=numpy.ndarray>

## Further Reading 

A more in-depth guide to working with Dask arrays can be found in the [dask tutorial](https://tutorial.dask.org/02_array.html).